<a href="https://www.kaggle.com/code/yassinezarwal/nn-housing?scriptVersionId=96561529" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


data_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col=['Id'])
data_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',index_col=['Id'])


# data split
train_x=data_train.drop(columns=['SalePrice'])
test_x=data_test
train_y=data_train.pop('SalePrice')

# categorical vs numeric data
ncolumns=train_x.select_dtypes(['int','float']).columns.tolist()
ccolumns=train_x.select_dtypes(['object']).columns.tolist()
train_x=train_x[ncolumns+ccolumns]


# cleaning data
cpipeline=Pipeline([('cimpute',SimpleImputer(strategy='most_frequent')),
                                        ('cencoder',OrdinalEncoder())])
npipeline=Pipeline([('nimpute',SimpleImputer(strategy='mean'))])

full_pipeline=ColumnTransformer([('num',npipeline,ncolumns),('cat',cpipeline,ccolumns)])


train_x=pd.DataFrame(full_pipeline.fit_transform(train_x),index=train_x.index,columns=train_x.columns)
test_x=pd.DataFrame(full_pipeline.fit_transform(test_x),index=test_x.index,columns=test_x.columns)

index_test=test_x.index

# scaling data
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

train_x=scaler.fit_transform(train_x)
test_x=scaler.fit_transform(test_x)



import tensorflow as tf 
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression


# split data
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y)

# creat two inputs for the model /using mutual_info_regression to separate inputs 
mir=mutual_info_regression(train_x,train_y)
columns_a=[i for i,e in enumerate(mutual_info_regression(train_x,train_y)) if e>0.3 ]
columns_b=[i for i,e in enumerate(mutual_info_regression(train_x,train_y)) if e<=0.3 ]

train_a,train_b=train_x[:,columns_a],train_x[:,columns_b]
valid_a,valid_b=valid_x[:,columns_a],valid_x[:,columns_b]
test_a,test_b=test_x[:,columns_a],test_x[:,columns_b]


# build a model/using the functional API
input_a=keras.layers.Input(shape=train_a.shape[1:])
input_b=keras.layers.Input(shape=train_b.shape[1:])

hidden1= keras.layers.Dense(79,activation='relu')(input_b)
hidden2=keras.layers.Dense(20,activation='relu')(hidden1)

concat=keras.layers.Concatenate()([input_a,hidden2])
output=keras.layers.Dense(1)(concat)

model=keras.models.Model(inputs=[input_a,input_b],outputs=[output])

sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error',optimizer=sgd,metrics=['accuracy'])



# # build a model/using the sequential API

# model=keras.models.Sequential([
#     keras.layers.Dense(79,activation='relu',input_shape=train_x.shape[1:]),
#     keras.layers.Dense(79,activation='relu'),
#     keras.layers.Dense(40,activation='relu'),
#     keras.layers.Dense(20,activation='relu'),
#     keras.layers.Dense(1)
# ])


# save the model
model.save('my_model.h5')

# using callbacks
checkpoint_cb=keras.callbacks.ModelCheckpoint('my_model.h5',save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)


# training the model
history=model.fit((train_a,train_b),train_y,epochs=500,validation_data=((valid_a,valid_b),valid_y),callbacks=[checkpoint_cb,early_stopping_cb])


# load the model
model=keras.models.load_model('my_model.h5')


# make prediction for test_data
predictions=model.predict((test_a,test_b))

predictions=pd.DataFrame(predictions,index=index_test,columns=['SalePrice'])

predictions.to_csv('gender_submission_housing26.csv')

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


2022-05-25 13:34:21.673558: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-05-25 13:34:21.673671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-25 13:34:32.876219: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-25 13:34:32.879620: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-05-25 13:34:32.879658: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-25 13:34:32.879693: I tensorflow/stream_executor/cuda/cud

Epoch 1/500
35/35 [==============================] - 1s 20ms/step - loss: 3799307344636131.5000 - accuracy: 0.0000e+00 - val_loss: 11566059618304.0000 - val_accuracy: 0.0000e+00
Epoch 2/500
35/35 [==============================] - 0s 3ms/step - loss: 8383097631175.1113 - accuracy: 0.0000e+00 - val_loss: 2889807822848.0000 - val_accuracy: 0.0000e+00
Epoch 3/500
35/35 [==============================] - 0s 3ms/step - loss: 1835558112824.8889 - accuracy: 0.0000e+00 - val_loss: 670044651520.0000 - val_accuracy: 0.0000e+00
Epoch 4/500
35/35 [==============================] - 0s 3ms/step - loss: 442662734506.6667 - accuracy: 0.0000e+00 - val_loss: 198589956096.0000 - val_accuracy: 0.0000e+00
Epoch 5/500
35/35 [==============================] - 0s 3ms/step - loss: 143700484323.5555 - accuracy: 0.0000e+00 - val_loss: 79667224576.0000 - val_accuracy: 0.0000e+00
Epoch 6/500
35/35 [==============================] - 0s 3ms/step - loss: 62408573838.2222 - accuracy: 0.0000e+00 - val_loss: 42355609600